In [ ]:
import pandas as pan
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plotter
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR